In [ ]:
from osgeo import gdal, ogr
dataset = gdal.Open(r'../input/eurosat-dataset/EuroSATallBands/Highway/Highway_1000.tif')


In [ ]:
!pip install rasterio

In [ ]:
import rasterio
import os
import numpy as np
import pandas as pd

In [ ]:
def load_image(filepath):
    image_sample = rasterio.open(str(filepath), "r")
    bands = [i for i in range(1, image_sample.count+1)]
    image_sample = image_sample.read(bands)
    image_sample = image_sample.astype('float32')
    #image_sample[np.isnan(image_sample)] = 0
    
    return image_sample

In [ ]:
im_ar = load_image('../input/eurosat-dataset/EuroSATallBands/Highway/Highway_1000.tif')

In [ ]:
im_ar.shape

In [ ]:
dataset_description = {}
dataset_description['imagename'] = []
dataset_description['Channels'] = []
dataset_description['Height'] = []
dataset_description['Width'] = []
dataset_description['Zero_non_zero_pixel_ratio'] = []
dataset_description['minPixelValue'] = []
dataset_description['maxPixelValue'] = []
dataset_description

In [ ]:
mypath = '../input/eurosat-dataset/EuroSATallBands'
subfolders = [os.path.join(mypath,folderi) for folderi in os.listdir(mypath) if '.' not in folderi]
subfolders

In [ ]:
list_of_files = []
for j in subfolders:
    list_of_files_temp = os.listdir(j)
    list_of_files_temp = [os.path.join(j, i) for i in list_of_files_temp if (i[-3:] == 'tif')]
    list_of_files.extend(list_of_files_temp)

In [ ]:
filelist = list_of_files

In [ ]:
for filename in filelist:
    image_sample = load_image(filename)
    zero_to_nz_ratio = np.isnan(image_sample).sum() / image_sample.size

    image_sample[np.isnan(image_sample)] = 0
    min_value = image_sample.min()
    max_value = image_sample.max()
    
    dataset_description['imagename'].append(filename)
    dataset_description['Channels'].append(image_sample.shape[0])
    dataset_description['Height'].append(image_sample.shape[1])
    dataset_description['Width'].append(image_sample.shape[2])
    dataset_description['Zero_non_zero_pixel_ratio'].append(zero_to_nz_ratio)
    dataset_description['minPixelValue'].append(min_value)
    dataset_description['maxPixelValue'].append(max_value)

In [ ]:
dataset_description_df = pd.DataFrame(dataset_description)

In [ ]:
dataset_description_df.to_csv('dataset_description_of_eurosat.csv')

In [ ]:
dataset_description_df.duplicated(subset=['imagename']).sum()

In [ ]:
dataset_description_df['class'] = None

In [ ]:
for i in range(0,dataset_description_df.shape[0]):
    class_name = dataset_description_df['imagename'][i].split('/')[-2]
    dataset_description_df['class'][i] = class_name

In [ ]:
dataset_description_df

In [ ]:
grps = dataset_description_df.groupby('class')

In [ ]:
grpnames = dataset_description_df['class'].unique().tolist()

In [ ]:
for i in grpnames:
    print(i)
    print(pd.DataFrame(grps.get_group(i)).describe())

In [ ]:
dataset_description_df

In [ ]:
dataset_description_df_new = dataset_description_df[dataset_description_df['class']!='Forest']
dataset_description_df_new

In [ ]:
dataset_description_df_new_2 = dataset_description_df_new[dataset_description_df_new['maxPixelValue']<5500]

In [ ]:
dataset_description_df_new_2['class'].value_counts()

In [ ]:
dataset_description_df_new_2.to_csv('selected_images_eurosat.csv')

In [ ]:
dataset_description_df_new_2

In [ ]:
dataset_description_df_new_2.duplicated(subset=['imagename']).sum()

In [ ]:
dataset_description_df_new_sampled = dataset_description_df_new_2.groupby("class").sample(n=1000, random_state=56)


In [ ]:
dataset_description_df_new_sampled.duplicated(subset=['imagename']).sum()

In [ ]:
dataset_description_df_new_sampled.describe()

In [ ]:
dataset_description_df_new_sampled['class'].value_counts()

In [ ]:
dataset_description_df_new_sampled.to_csv('./selected_images_eurosat.csv')

In [ ]:
im_ar = load_image('../input/eurosat-dataset/EuroSATallBands/Highway/Highway_1000.tif')

In [ ]:
im_ar.shape

In [ ]:
im_ar_t = im_ar.transpose((1, 2, 0))

In [ ]:
im_ar_t.shape

In [ ]:
filepath = '../input/eurosat-dataset/EuroSATallBands/Highway/Highway_1000.tif'